In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
home_page = pd.read_csv("home_page_table.csv")
home_page.sample(3)

In [ ]:
payment_confirmation = pd.read_csv("payment_confirmation_table.csv")
payment_confirmation.sample(3)

In [ ]:
payment_page = pd.read_csv("payment_page_table.csv")
payment_page.sample(3)

In [ ]:
search_page = pd.read_csv("search_page_table.csv")
search_page.sample(3)

In [ ]:
user = pd.read_csv("user_table.csv")
user.sample(3)

home page -> search page -> payment page -> confirmation page

In [ ]:
df = pd.merge(left=user, right=home_page, how='left', on='user_id')
df = pd.merge(left=df, right=search_page, how='left', on='user_id', suffixes=('_home', '_search'))
df = pd.merge(left=df, right=payment_page, how='left', on='user_id')
df = pd.merge(left=df, right=payment_confirmation, how='left', on='user_id', suffixes=('_payment', '_confirmation'))
df.head()

In [ ]:
df.info()

In [ ]:
# Calculate the number of users at each step
steps = ["page_home", "page_search", "page_payment", "page_confirmation"]
funnel_counts = [df[step].notnull().sum() for step in steps]

# Plot the funnel chart
plt.figure(figsize=(8, 5))
plt.barh(steps[::-1], funnel_counts[::-1], color='skyblue')  # Reverse order for proper funnel visualization
plt.xlabel("Number of Users")
plt.title("Funnel Visualization")
plt.show()

In [ ]:
data = (
    df.groupby(["device", "sex"])[["page_home", "page_search", "page_payment", "page_confirmation"]]
    .count()
    .reset_index()
    .assign(
        home_to_search=lambda x: (x["page_search"] / x["page_home"]) * 100,
        search_to_payment=lambda x: (x["page_payment"] / x["page_search"]) * 100,
        payment_to_confirmation=lambda x: (x["page_confirmation"] / x["page_payment"]) * 100,
    )
    .melt(
        id_vars=['device', 'sex'],
        value_vars=['home_to_search', 'search_to_payment', 'payment_to_confirmation'],
        var_name='conversion_step',
        value_name='conversion_rate',
    )
)

# Plot the conversion rates
fig, ax = plt.subplots(figsize=(15, 5))
ax = sns.barplot(data=data, x="conversion_step", y="conversion_rate", hue="device", palette="Set2")

for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height() + 2
    ax.text(x=x, y=y, s=f"{p.get_height():.2f}%", ha="center", va="bottom", fontsize=10)

# Add labels and title
ax.set_title("Conversion Rates by Device", fontsize=16)
ax.set_ylabel("Conversion Rate (%)", fontsize=14)
ax.set_ylim(0, 100)
ax.legend(
    title="Device",
    fontsize=12,
    loc="upper left",  # Anchor the legend to the upper left
    bbox_to_anchor=(1.01, 1),  # Place it just outside the right side of the plot
)


# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
data = (
    df.groupby(["device", "sex"])[["page_home", "page_search", "page_payment", "page_confirmation"]]
    .count()
    .reset_index()
    .assign(
        home_to_search=lambda x: (x["page_search"] / x["page_home"]) * 100,
        search_to_payment=lambda x: (x["page_payment"] / x["page_search"]) * 100,
        payment_to_confirmation=lambda x: (x["page_confirmation"] / x["page_payment"]) * 100,
    )
    .melt(
        id_vars=['device', 'sex'],
        value_vars=['home_to_search', 'search_to_payment', 'payment_to_confirmation'],
        var_name='conversion_step',
        value_name='conversion_rate',
    )
)

# Plot the conversion rates
fig, ax = plt.subplots(figsize=(15, 5))
ax = sns.barplot(data=data, x="conversion_step", y="conversion_rate", hue="sex", palette="Set2")

for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height() + 2
    ax.text(x=x, y=y, s=f"{p.get_height():.2f}%", ha="center", va="bottom", fontsize=10)

# Add labels and title
ax.set_title("Conversion Rates by Sex", fontsize=16)
ax.set_ylabel("Conversion Rate (%)", fontsize=14)
ax.set_ylim(0, 100)
ax.legend(
    title="Sex",
    fontsize=12,
    loc="upper left",  # Anchor the legend to the upper left
    bbox_to_anchor=(1.01, 1),  # Place it just outside the right side of the plot
)


# Show the plot
plt.tight_layout()
plt.show()



In [ ]:
# Calculate conversion rates step by step
df['home_to_search'] = (df['page_search'] / df['page_home']) * 100
df['search_to_payment'] = (df['page_payment'] / df['page_search']) * 100
df['payment_to_confirmation'] = (df['page_confirmation'] / df['page_payment']) * 100


In [ ]:
# Calculate the number of users at each step
steps = ["page_home", "page_search", "page_payment", "page_confirmation"]
funnel_counts = [df[step].notnull().sum() for step in steps]

# Create a Plotly funnel chart
fig = go.Figure(
    go.Funnel(
        y=steps,  # Steps of the funnel
        x=funnel_counts,  # Counts at each step
        textinfo="value+percent initial",  # Show value and percentage
        # opacity = 0.65,
        marker={"color": ["yellow", "gold", "lightgreen", "lightblue"], "opacity": 0.85},  # Customize colors
        # connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}},
    )
)

# Customize the layout
fig.update_layout(
    title="Overall Conversion Funnel",
    xaxis_title="Number of Users",
    yaxis_title="Funnel Steps",
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot background
    # paper_bgcolor="rgba(0,0,0,0)"  # Transparent chart background
)

# Display the funnel chart
fig.show()

From Payment Page to Confirmation Page: A sharp decline here indicates potential issues in the payment process, causing users to abandon their transactions.

In [ ]:
# Calculate the number of users at each step
steps = ["page_home", "page_search", "page_payment", "page_confirmation"]

fig = go.Figure()
# Create a Plotly funnel chart
fig.add_trace(
    go.Funnel(
        name="Desktop",
        y=steps,  # Steps of the funnel
        x=[df[df["device"]=='Desktop'][step].notnull().sum() for step in steps],  # Counts at each step
        textinfo="value+percent previous",  # Show value and percentage
        # marker={"color": ["yellow", "gold", "lightgreen", "lightblue"], "opacity": 0.85},  # Customize colors
    )
)

fig.add_trace(
    go.Funnel(
        name="Mobile",
        y=steps,  # Steps of the funnel
        # orientation="h",
        x=[df[df["device"]=='Mobile'][step].notnull().sum() for step in steps],  # Counts at each step
        textinfo="value+percent previous",  # Show value and percentage
        # marker={"color": ["yellow", "gold", "lightgreen", "lightblue"], "opacity": 0.85},  # Customize colors
    )
)


# Customize the layout
fig.update_layout(
    title="Conversion Funnel by Device",
    funnelmode="stack",  # Use stack mode
    xaxis_title="Number of Users",
    yaxis_title="Funnel Steps",
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot background
    # paper_bgcolor="rgba(0,0,0,0)"  # Transparent chart background
)

# Display the funnel chart
fig.show()

Both desktop and mobile users exhibit similar drop-off patterns, implying that the issues are platform-agnostic and may relate to overall user experience or process design.

**Potential Metrics to Track**
- Search Success Rate:
    - Measure the percentage of users who find relevant results and proceed to the next step.
- Payment Completion Rate:
    - Track how many users successfully complete the payment process.
- Session Duration:
    - Analyze time spent on key stages to identify friction points.
- Bounce Rates:
    - Look at where users are dropping off and optimize those stages.

**Streamline the Payment Process**

- Optimize payment forms:
    - Minimize the number of fields and steps required for payment.
    - Include a progress bar to show users how many steps are left.
- Build trust:
    - Highlight security measures (e.g., "Secure Payment" badges).
    - Offer multiple payment methods to cater to user preferences.
- Identify friction points:
    - Monitor payment failures and error messages to address issues quickly.

In [ ]:
# Calculate the number of users at each step
steps = ["page_home", "page_search", "page_payment", "page_confirmation"]

fig = go.Figure()
# Create a Plotly funnel chart
fig.add_trace(
    go.Funnel(
        name="Male",
        y=steps,  # Steps of the funnel
        x=[df[df["sex"]=='Male'][step].notnull().sum() for step in steps],  # Counts at each step
        textinfo="value+percent previous",  # Show value and percentage
        # marker={"color": ["yellow", "gold", "lightgreen", "lightblue"], "opacity": 0.85},  # Customize colors
    )
)

fig.add_trace(
    go.Funnel(
        name="Female",
        y=steps,  # Steps of the funnel
        # orientation="h",
        x=[df[df["sex"]=='Female'][step].notnull().sum() for step in steps],  # Counts at each step
        textinfo="value+percent previous",  # Show value and percentage
        # marker={"color": ["yellow", "gold", "lightgreen", "lightblue"], "opacity": 0.85},  # Customize colors
    )
)


# Customize the layout
fig.update_layout(
    title="Conversion Funnel by Sex",
    funnelmode="stack",  # Use stack mode
    xaxis_title="Number of Users",
    yaxis_title="Funnel Steps",
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot background
    # paper_bgcolor="rgba(0,0,0,0)"  # Transparent chart background
)

# Display the funnel chart
fig.show()

In [ ]:
data = (
    df.groupby(['device', 'sex'])[['page_home', 'page_search', 'page_payment', 'page_confirmation']]
    .count()
    .reset_index()
    .melt(
        id_vars=['device', 'sex'], 
        value_vars=['page_home', 'page_search', 'page_payment', 'page_confirmation'], 
        var_name='funnel_steps', 
        value_name='funnel_counts'
    )
)

In [ ]:
# Filter data for Desktop
device = "Desktop"
device_data = data[data["device"] == device]

# Create the figure
fig = go.Figure()

# Add Male trace
male_data = device_data[device_data["sex"] == "Male"]
fig.add_trace(
    go.Funnel(
        name="Male",
        y=male_data["funnel_steps"],  # Steps of the funnel
        x=male_data["funnel_counts"],  # Counts at each step
        textinfo="value+percent initial",  # Show value and percentage
        textposition="inside",  # Text inside the bars
        textfont=dict(size=16, color="white"),  # Font size and color
        marker=dict(color="rgba(54, 162, 235, 0.9)")  # Use semi-transparent blue
    )
)

# Add Female trace
female_data = device_data[device_data["sex"] == "Female"]
fig.add_trace(
    go.Funnel(
        name="Female",
        y=female_data["funnel_steps"],  # Steps of the funnel
        x=female_data["funnel_counts"],  # Counts at each step
        textinfo="value+percent initial",  # Show value and percentage
        textposition="inside",  # Text inside the bars
        textfont=dict(size=16, color="white"),  # Font size and color
        marker=dict(color="rgba(255, 99, 132, 0.9)")  # Use semi-transparent red
    )
)

# Update layout for improved appearance
fig.update_layout(
    title=dict(
        text=f"Conversion Funnel for {device}",
        font=dict(size=20, color="black"),  # Title font size and color
        x=0.5  # Center the title
    ),
    funnelmode="stack",  # Use stack mode
    width=1500,  # Adjust width
    height=400,  # Adjust height
    plot_bgcolor="white",  # Set plot background color to white
    paper_bgcolor="white",  # Set chart (paper) background color to white
    margin=dict(l=50, r=50, t=80, b=50)  # Add margins for spacing
)

# Add legend customization
fig.update_layout(
    legend=dict(
        title="Gender",
        font=dict(size=12),
        orientation="h",  # Place legend horizontally
        x=0.5,
        xanchor="center",
        y=-0.1
    )
)

# Add connector customization
fig.update_traces(
    connector={"line": {"color": "gray", "dash": "dot", "width": 1.5}}
)

# Display the chart
fig.show()

In [ ]:
# Filter data for Desktop
device = "Mobile"
device_data = data[data["device"] == device]

# Create the figure
fig = go.Figure()

# Add Male trace
male_data = device_data[device_data["sex"] == "Male"]
fig.add_trace(
    go.Funnel(
        name="Male",
        y=male_data["funnel_steps"],  # Steps of the funnel
        x=male_data["funnel_counts"],  # Counts at each step
        textinfo="value+percent initial",  # Show value and percentage
        textposition="inside",  # Text inside the bars
        textfont=dict(size=16, color="white"),  # Font size and color
        marker=dict(color="rgba(54, 162, 235, 0.9)")  # Use semi-transparent blue
    )
)

# Add Female trace
female_data = device_data[device_data["sex"] == "Female"]
fig.add_trace(
    go.Funnel(
        name="Female",
        y=female_data["funnel_steps"],  # Steps of the funnel
        x=female_data["funnel_counts"],  # Counts at each step
        textinfo="value+percent initial",  # Show value and percentage
        textposition="inside",  # Text inside the bars
        textfont=dict(size=16, color="white"),  # Font size and color
        marker=dict(color="rgba(255, 99, 132, 0.9)")  # Use semi-transparent red
    )
)

# Update layout for improved appearance
fig.update_layout(
    title=dict(
        text=f"Stacked Funnel Chart for {device}",
        font=dict(size=20, color="black"),  # Title font size and color
        x=0.5  # Center the title
    ),
    funnelmode="stack",  # Use stack mode
    width=1500,  # Adjust width
    height=400,  # Adjust height
    plot_bgcolor="white",  # Set plot background color to white
    paper_bgcolor="white",  # Set chart (paper) background color to white
    margin=dict(l=50, r=50, t=80, b=50)  # Add margins for spacing
)

# Add legend customization
fig.update_layout(
    legend=dict(
        title="Gender",
        font=dict(size=12),
        orientation="h",  # Place legend horizontally
        x=0.5,
        xanchor="center",
        y=-0.1
    )
)

# Add connector customization
fig.update_traces(
    connector={"line": {"color": "gray", "dash": "dot", "width": 1.5}}
)

# Display the chart
fig.show()